In [1]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///db/stock_data.db")
print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM stock_data LIMIT 5;")}')

Dialect: sqlite
Available tables: ['stock_data']
Sample output: [('AMZN', '2025-08-19 00:00:00', 228.01, 29891010, 230.09, 230.5283, 227.12), ('AMZN', '2025-08-18 00:00:00', 231.49, 25248890, 230.225, 231.91, 228.33), ('AMZN', '2025-08-15 00:00:00', 231.03, 39649240, 232.58, 234.08, 229.807), ('AMZN', '2025-08-14 00:00:00', 230.98, 61545820, 227.4, 233.11, 227.02), ('AMZN', '2025-08-13 00:00:00', 224.56, 36508340, 222.0, 224.9185, 222.0)]


In [19]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-5-mini")

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

In [21]:
from langgraph.prebuilt import create_react_agent

system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

agent = create_react_agent(
    llm,
    tools,
    prompt=system_prompt,
)

In [ ]:
question = "Which stock has the highest close price?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
import gradio as gr
def chat(message, history):
    messages = history + [{"role": "user", "content": message}]
    stream=agent.stream({"messages":messages},stream_mode="values")
    response = ""
    for chunk in stream:
        response +=chunk["messages"][-1] or ''
        yield response

gr.ChatInterface(chat, type="messages").launch()

In [22]:
async def chat_with_agent(message, history):
    # LangGraph agents expect messages in a specific format
    messages = [{"role": "user", "content": message}]
    
    # Invoke the agent
    response = await agent.ainvoke({"messages": messages})
    
    # Extract the agent's reply
    agent_reply = response["messages"][-1].content
    return agent_reply

# 5. Create the Gradio interface
iface = gr.ChatInterface(
    fn=chat_with_agent,
    title="LangGraph ReAct Agent Chatbot",
    description="Ask me about stock questions!",
)

# 6. Launch the Gradio app
if __name__ == "__main__":
    iface.launch()

C:\Users\wyp_1\Projects\Upwork\financial_agentic_AI\.venv\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.
